In [ ]:
# https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/

In [ ]:
# python/tensorflow 워닝 막기
import warnings , os
#warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='once')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
# # # https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-17%EC%9D%BC%EC%B0%A8-%EB%A1%9C%EC%9D%B4%ED%84%B0-%EB%89%B4%EC%8A%A4-%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0-f5466abe9196
# # np.load()에서 import_pickle=False오류 날경우, 최초실행시 한번 설치해, 주석처리하고 재실행하면 downgrade로 적용됨
# !pip uninstall numpy
# !pip install --upgrade numpy==1.16.1

In [ ]:
import numpy as np
print(np.__version__) #'1.16.1'

1.16.1


In [ ]:
###########################
# google drive 저장/로딩
###########################
#구글 드라이브 저장
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gcp_drive = None
def doGoodleDriveAuth():
    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    print('gauth:',gauth)
    gauth.credentials = GoogleCredentials.get_application_default()

    gcp_drive = GoogleDrive(gauth)
    print('gcp_drive:', gcp_drive)
    return gcp_drive

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html
def gcp_upload(file_path, w_file_name):
    try:
        if gcp_drive == None:
            drive = doGoodleDriveAuth()

        # 특정 폴더 안으로 파일 삽입
        uploaded = drive.CreateFile({'title': w_file_name}) #, "parents": [{"kind": "drive#fileLink","id": 'jukyellow@gmail.com'}]})
        uploaded.SetContentString(w_file_name)
        uploaded.SetContentFile(file_path + w_file_name)
        uploaded.Upload()
        print('Uploaded file with ID {}'.format(uploaded.get('id')))
        return uploaded.get('id')
    except Exception as e: print('gcp_upload err:', e)

#구글 드라이브에서 다운로드
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
import io
from io import BytesIO 
from googleapiclient.http import MediaIoBaseDownload

TEMP_PATH = '/tmp/'
def gcp_download(file_name, key):
    #3. 모델 다운로드
    #https://drive.google.com/open?id=1TlvbayGRCjAI6bOZrUYMmv6g6b95rnRM
    request = drive_service.files().get_media(fileId=key)

    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      if status:
          print("Download %%%d%%." % int(status.progress() * 100))
      print("Download Complete!")
    downloaded.seek(0)

    with open(TEMP_PATH + file_name, 'wb') as f:
        f.write(downloaded.read())

/usr/local/lib/python3.6/dist-packages/google/colab/auth.py:148: ResourceWarning: unclosed <ssl.SSLSocket fd=61, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('172.28.0.2', 42174), raddr=('172.217.13.74', 443)>
  if not _check_adc():
/usr/local/lib/python3.6/dist-packages/google/colab/auth.py:169: ResourceWarning: unclosed <ssl.SSLSocket fd=61, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('172.28.0.2', 42176), raddr=('172.217.13.74', 443)>
  if _check_adc():
/usr/local/lib/python3.6/dist-packages/googleapiclient/_helpers.py:134: ResourceWarning: unclosed <ssl.SSLSocket fd=61, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('172.28.0.2', 35236), raddr=('172.217.9.202', 443)>
  return wrapped(*args, **kwargs)


In [ ]:
# https://drive.google.com/file/d/1_Hj9zu0lYUoeL0d1AqPGZa78OxNrpER2/view?usp=sharing
# gcp에서 폴더명 사진 다운로드, 압축풀기
down_file_name = 'facenet_keras.h5'
gcp_download(down_file_name, '1_Hj9zu0lYUoeL0d1AqPGZa78OxNrpER2') 

Download %100%.
Download Complete!


In [ ]:
!mv /tmp/facenet_keras.h5 /content

In [ ]:
# gcp에서 폴더명 사진 다운로드, 압축풀기
down_file_name = '00_origin_전체.zip'
gcp_download(down_file_name, '1k0lm-uBfxlUbqtzvZ4xjVa_ojS0fPFUG')  #https://drive.google.com/file/d/1k0lm-uBfxlUbqtzvZ4xjVa_ojS0fPFUG/view?usp=sharing

Download %100%.
Download Complete!


In [ ]:
!\rm -rf /content/origin_data
!\rm -rf /content/face_data

!mkdir /content/origin_data

In [ ]:
# 한글깨짐 방지: -O cp949 
!unzip -O cp949 /tmp/00_origin_전체.zip
!rm -rf /tmp/00_origin_전체.zip

!mv /content/00_전체 /content/origin_data

!mkdir /content/face_data
!mkdir /content/face_data/train
!mkdir /content/face_data/test

In [ ]:
# 원본경로 파일목록 읽기
import os

path = '/content/origin_data/00_전체'
imgFileList = os.listdir(path)

imgFileList = [x for x in imgFileList if 'jpg' in x or 'jpeg' in x] # jpg파일만 불러오기

for i, file_name in enumerate(imgFileList):
  print('i:',i+1 , ' ,name:', file_name)
  #if i > 1 : break

In [ ]:
# preview 이미지 생성 메소드 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def save_img_by_img_gen(path, file_name, MAX_GEN_CNT, save_path, new_dir, gen_type):
    if "train" == gen_type:
        datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            #rescale=1./255, predict시에는 이미지 스케일 작업없어서 제외
            shear_range=0.1,
            zoom_range=0.1, #확대/축소 비율
            brightness_range= [0.1, 1.0], # 밝기 비율
            channel_shift_range=80, # rgb 채널 랜덤 섞기(명암 어둡게 변함)
            horizontal_flip=True,  #가로 대칭 전환
            cval = 255, # constant일때 여백 흰색
            fill_mode='constant') #nearest constant
    else:
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            #rescale=1./255, predict시에는 이미지 스케일 작업없어서 제외
            shear_range=0.2,
            zoom_range=0.2, #확대/축소 비율
            brightness_range= [0.1, 1.0], # 밝기 비율
            channel_shift_range=80, # rgb 채널 랜덤 섞기(명암 어둡게 변함)
            horizontal_flip=True,  #가로 대칭 전환
            cval = 255, # constant일때 여백 흰색
            fill_mode='constant') #nearest constant

    img = load_img(path+"/"+file_name)  # PIL 이미지
    x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
    x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열
    print('x.shape:', x.shape)

    #파일명별 폴더 생성
    file_name = file_name.replace(".jpg","")
    new_path = new_dir + file_name
    if not(os.path.isdir(new_path)): os.makedirs(os.path.join(new_path))

    # 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서, save_to_dir저장
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=save_path+file_name, save_prefix=file_name, save_format='jpg'):
        i += 1
        if i > MAX_GEN_CNT: break 

# 이미지 변환 파일 생성
MAX_GEN_CNT = 20
for i, file_name in enumerate(imgFileList):
    save_img_by_img_gen(path, file_name, MAX_GEN_CNT, 'face_data/train/', '/content/face_data/train/', 'train')
    print('i:',i+1 , ' ,name:', file_name + " 변형 완료!")

In [ ]:
# 이미지 변환 파일 생성
MAX_GEN_CNT = 10
for i, file_name in enumerate(imgFileList):
    save_img_by_img_gen(path, file_name, MAX_GEN_CNT, 'face_data/test/', '/content/face_data/test/', 'test')
    print('i:',i+1 , ' ,name:', file_name + " 변형 완료!")

In [ ]:
# example of loading the keras facenet model
from keras.models import load_model
# load the model
facenet_model = load_model('facenet_keras.h5')
# summarize input and output shape
print(facenet_model.inputs)
print(facenet_model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [ ]:
!pip install mtcnn

In [ ]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [ ]:
ls -alrt

total 91180
drwxr-xr-x 1 root root     4096 Dec 21 17:29 sample_data/
drwxr-xr-x 1 root root     4096 Jan  6 06:26 ../
drwxr-xr-x 1 root root     4096 Jan  6 06:38 .config/
-rw-r--r-- 1 root root      720 Jan  6 06:38 adc.json
-rw-r--r-- 1 root root   456511 Jan  6 06:54 faces-embeddings.npz
-rw-r--r-- 1 root root   456809 Jan  6 07:04 faces-dataset.npz
-rw-r--r-- 1 root root     8491 Jan  6 07:05 web_cam_capture_1.jpg
-rw-r--r-- 1 root root 92397640 Jan  6 07:12 facenet_keras.h5
drwxr-xr-x 3 root root     4096 Jan  6 07:13 origin_data/
drwxr-xr-x 1 root root     4096 Jan  6 07:13 ./
drwxr-xr-x 4 root root     4096 Jan  6 07:13 face_data/


In [ ]:
# demonstrate face detection on 5 Celebrity Faces Dataset
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    if len(results) == 0 :
        print('------------------------------------------')
        print('results:', results, ', filename:', filename)
        print('------------------------------------------')
        return []

    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
# specify folder to plot
folder = './face_data/test/B05_OOO/'
i = 1
# enumerate files
for filename in listdir(folder):
    if 'jpg' not in filename and 'jpeg' not in filename: continue
    # path
    path = folder + filename
    # get face
    face = extract_face(path)

    if len(face) != 0 :
        print(i, face.shape)
        # plot
        pyplot.subplot(2, 7, i)
        pyplot.axis('off')
        pyplot.imshow(face)
        i += 1
    else: print('extract_face err:', i)
pyplot.show()

In [ ]:
# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)

    if len(results) == 0 :
        print('------------------------------------------')
        print('results:', results, ', filename:', filename)
        print('------------------------------------------')
        return []

    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        if 'jpg' not in filename and 'jpeg' not in filename: continue

        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        if len(face) != 0:
            # store
            faces.append(face)
        else: print('extract_face err')
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    cnt = 0
    for subdir in listdir(directory):
        cnt += 1
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
          continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>cnt:', cnt, ' ,loaded %d examples for class: %s' % (len(faces), subdir), ' ,labels:', labels)
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

import time
start = time.time() # 시작 시간 저장

# load train dataset
trainX, trainy = load_dataset('./face_data/train/')
print('trainX.shape:', trainX.shape)
print('trainy.shape:', trainy.shape)

print("time :", time.time() - start) # 현재시각 - 시작시간 = 실행 시간


In [ ]:
# 작업 코드
print("time :", time.time() - start) # 현재시각 - 시작시간 = 실행 시간

# load test dataset
testX, testy = load_dataset('./face_data/test/')
print('testX.shape:', testX.shape)
print('testy.shape:', testy.shape)
print(testy)

print("time :", time.time() - start) # 현재시각 - 시작시간 = 실행 시간

In [ ]:
# save arrays to one file in compressed format
savez_compressed('faces-dataset.npz', trainX, trainy, testX, testy)

In [ ]:
ls -alrt

total 147252
drwxr-xr-x 1 root root     4096 Dec 21 17:29 sample_data/
drwxr-xr-x 1 root root     4096 Jan  6 06:26 ../
drwxr-xr-x 1 root root     4096 Jan  6 06:38 .config/
-rw-r--r-- 1 root root      720 Jan  6 06:38 adc.json
-rw-r--r-- 1 root root   456511 Jan  6 06:54 faces-embeddings.npz
-rw-r--r-- 1 root root     8491 Jan  6 07:05 web_cam_capture_1.jpg
-rw-r--r-- 1 root root 92397640 Jan  6 07:12 facenet_keras.h5
drwxr-xr-x 3 root root     4096 Jan  6 07:13 origin_data/
drwxr-xr-x 1 root root     4096 Jan  6 07:13 ./
drwxr-xr-x 4 root root     4096 Jan  6 07:13 face_data/
-rw-r--r-- 1 root root 57874962 Jan  6 07:27 faces-dataset.npz


In [ ]:
print('trainX:', trainX.shape)
print('trainy:', trainy.shape)
print('testX:', testX.shape)
print('testy:', testy.shape)

trainX: (630, 160, 160, 3)
trainy: (630,)
testX: (328, 160, 160, 3)
testy: (328,)


In [ ]:
# calculate a face embedding for each face in the dataset using facenet
import numpy as np
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

# #https://d-tail.tistory.com/31
# ## 먼저 기존의 np.load를 np_load_old에 저장해둠.
# np_load_old = np.load
# ## 기존의 parameter을 바꿔줌
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

# load the face dataset
data = load('faces-dataset.npz')
#print('data:', data)
#for item in data: print(data[item])

print('trainX:', data['arr_0'].shape)
print('trainy:', data['arr_1'].shape)
print('testX:', data['arr_2'].shape)
print('testy:', data['arr_3'].shape)

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)


trainX: (630, 160, 160, 3)
trainy: (630,)
testX: (328, 160, 160, 3)
testy: (328,)
Loaded:  (630, 160, 160, 3) (630,) (328, 160, 160, 3) (328,)


In [ ]:
# load the facenet model
facenet_model = load_model('./facenet_keras.h5')
print('Loaded Model')

# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(facenet_model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

# convert each face in the test set to an embedding
newTestX = list()
for idx, face_pixels in enumerate(testX):
    if len(face_pixels)==0: 
        print('idx:', idx)
        continue
    embedding = get_embedding(facenet_model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

# save arrays to one file in compressed format
savez_compressed('faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

Loaded Model
(630, 128)
(328, 128)


In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load dataset
data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

# normalize input vectors
in_encoder = Normalizer(norm='l2')
print('trainX.shape:',trainX.shape)
print('trainX[0].shape:',trainX[0].shape)
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
print('trainX.shape:', trainX.shape)
print('testX.shape:', testX.shape)

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
print('trainy.shape:', trainy.shape)
print('testy.shape:', testy.shape)

print('testX:', testX)
print('testy:', testy)

Dataset: train=630, test=328
trainX.shape: (630, 128)
trainX[0].shape: (128,)
trainX.shape: (630, 128)
testX.shape: (328, 128)
trainy.shape: (630,)
testy.shape: (328,)
testX: [[ 0.06719897 -0.00746293 -0.07132015 ... -0.10519774  0.03399387
  -0.04146887]
 [ 0.11998394 -0.06810085 -0.08123402 ... -0.09245402 -0.0454261
  -0.05718428]
 [ 0.05698761  0.02405528 -0.10154805 ... -0.0466873   0.10299142
  -0.02957636]
 ...
 [ 0.06540772 -0.08778912 -0.01173866 ... -0.18333246 -0.02484478
  -0.04982429]
 [ 0.06045805 -0.11666107  0.0163904  ... -0.15685587  0.00355646
  -0.0986824 ]
 [ 0.10205339 -0.09236851 -0.07657373 ... -0.17265351 -0.05783626
  -0.05999085]]
testy: [13 13 13 13 13 13 13 13 13 13 19 19 19 19 19 19 19 19 19 19 19 25 25 25
 25 25 25 25 25 25 25 25 23 23 23 23 23 23 23 23 23 23 23 21 21 21 21 21
 21 21 21 21 21 21  2  2  2  2  2  2  2  2  2  2  2 22 22 22 22 22 22 22
 22 22 22 22 11 11 11 11 11 11 11 11 11 11 11  0  0  0  0  0  0  0  0  0
  0  0 24 24 24 24 24 24 24 24 24 2

In [ ]:
# fit model
pred_model = SVC(kernel='linear', probability=True)
pred_model.fit(trainX, trainy)

# predict
yhat_train = pred_model.predict(trainX)
yhat_test = pred_model.predict(testX)

# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=100.000


In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot

# load faces
data = load('faces-dataset.npz')
testX_faces = data['arr_2']

# load face embeddings
data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('trainX.shape:', trainX.shape)

# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

# fit model
pred_model = SVC(kernel='linear', probability=True)
pred_model.fit(trainX, trainy)

# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = pred_model.predict(samples)
yhat_prob = pred_model.predict_proba(samples)

# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])

# plot for fun
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()

In [ ]:
print(testy)

In [ ]:
path="./web_cam_capture_1.jpg"
face_pixels = extract_face(path)
print('face:', face_pixels.shape)

embedding = get_embedding(facenet_model, face_pixels)
print('embedding:', embedding.shape)

in_encoder = Normalizer(norm='l2')
#dataX = in_encoder.transform(np.array([embedding]))
dataX = in_encoder.transform([embedding])
print('dataX:', dataX.shape)

# # test model on a random example from the test dataset
# selection = choice([i for i in range(testX.shape[0])])
# random_face_pixels = testX_faces[selection]
# random_face_emb = testX[selection]
# random_face_class = testy[selection]
# random_face_name = out_encoder.inverse_transform([random_face_class])

# prediction for the face
#dataX = expand_dims(dataX, axis=0)
#print('dataX:', dataX.shape)
yhat_class = pred_model.predict(dataX)
yhat_prob = pred_model.predict_proba(dataX)

# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
# print('Expected: %s' % random_face_name[0])

# plot for fun
pyplot.imshow(face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()